<a href="https://colab.research.google.com/github/Devraj02-sys/Dev/blob/main/Machine%20learning%20model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from sklearn.datasets import fetch_california_housing
housing=fetch_california_housing()

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_california_housing
import pandas as pd

housing=fetch_california_housing()

param_grid = [
        {'svr__kernel': ['linear'], 'svr__C': [10., 30., 100., 300., 1000.,
                                               3000., 10000., 30000.0]},
        {'svr__kernel': ['rbf'], 'svr__C': [1.0, 3.0, 10., 30., 100., 300.,
                                            1000.0],
         'svr__gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0]},
    ]
preprocessing=StandardScaler()
create_pipeline=Pipeline([('preprocessing',preprocessing),('svr',SVR())])
housing_df=pd.DataFrame(housing.data,columns=housing.feature_names)
housing_target_df=pd.DataFrame(housing.target,columns=['target'])
housing_df=pd.concat([housing_df,housing_target_df],axis=1)
grid_search=GridSearchCV(create_pipeline,param_grid,cv=5,scoring='neg_mean_squared_error')
grid_search.fit(housing_df.iloc[:5000],housing_target_df.iloc[:5000].values.ravel())

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing', StandardScaler()),
                                       ('svr', SVR())]),
             param_grid=[{'svr__C': [10.0, 30.0, 100.0, 300.0, 1000.0, 3000.0,
                                     10000.0, 30000.0],
                          'svr__kernel': ['linear']},
                         {'svr__C': [1.0, 3.0, 10.0, 30.0, 100.0, 300.0,
                                     1000.0],
                          'svr__gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0],
                          'svr__kernel': ['rbf']}],
             scoring='neg_mean_squared_error')

In [15]:
svr_grid_search_rmse=-grid_search.best_score_
svr_grid_search_rmse
# grid_search.best_params_

0.0033322804942019476

In [23]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import expon, loguniform
param_distribs = {
        'svr__kernel': ['linear', 'rbf'],
        'svr__C': loguniform(20, 200_000),
        'svr__gamma': expon(scale=1.0),
    }

rnd_search = RandomizedSearchCV(create_pipeline,
                                param_distributions=param_distribs,
                                n_iter=50, cv=5,
                                scoring='neg_root_mean_squared_error',
                                random_state=42)
rnd_search.fit(housing_df.iloc[:5000], housing_target_df.iloc[:5000].values.ravel())

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessing',
                                              StandardScaler()),
                                             ('svr', SVR())]),
                   n_iter=50,
                   param_distributions={'svr__C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7ac3e1d7ed50>,
                                        'svr__gamma': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7ac3e0919f50>,
                                        'svr__kernel': ['linear', 'rbf']},
                   random_state=42, scoring='neg_root_mean_squared_error')

In [24]:
svr_rnd_search_rmse = -rnd_search.best_score_
svr_rnd_search_rmse

0.05659153696693666

In [14]:
rnd_search.best_params_

{'svr__C': 629.7823295913721,
 'svr__gamma': 3.010121430917521,
 'svr__kernel': 'linear'}

In [21]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
selector_pipeline = Pipeline([
    ('preprocessing', preprocessing),
    ('selector', SelectFromModel(RandomForestRegressor(random_state=42),
                                 threshold=0.005)),  # min feature importance
    ('svr', SVR(C=rnd_search.best_params_["svr__C"],
                gamma=rnd_search.best_params_["svr__gamma"],
                kernel=rnd_search.best_params_["svr__kernel"])),
])
selector_pipeline

Pipeline(steps=[('preprocessing', StandardScaler()),
                ('selector',
                 SelectFromModel(estimator=RandomForestRegressor(random_state=42),
                                 threshold=0.005)),
                ('svr',
                 SVR(C=629.7823295913721, gamma=3.010121430917521,
                     kernel='linear'))])

In [22]:
from sklearn.model_selection import cross_val_score
selector_rmses = -cross_val_score(selector_pipeline,
                                  housing_df.iloc[:5000],
                                  housing_target_df.iloc[:5000],
                                  scoring="neg_root_mean_squared_error",
                                  cv=3)
pd.Series(selector_rmses).describe()

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for 

,0
count,3.000000
mean,0.057232
std,0.017886
min,0.046652
25%,0.046907
50%,0.047162
75%,0.062523
max,0.077883
